# Test Playground: A place to try out our ideas in a notebook!

This is a place where we want to try out new ideas with test scripts for data analysis of FED-related data.

### Import libraries of interest

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import os
pd.set_option('display.max_rows', None)

# This requires Python 3.7 and above

In [2]:
def read_csv_files(file_path):
        # reads csv files and organizes them into dataframes
        
        all_dataframes = []
        # for all files in folder
        for file in os.listdir(file_path):
            if file.endswith(".CSV"):
                if file.startswith("FED"):
                    # read that file into a dataframe
                    file_path_df = os.path.join(file_path ,file)
                    df = pd.read_csv(file_path_df)
                    all_dataframes.append(df)        
                              
        # create a single dataframe from all files
        singular_df = pd.concat(all_dataframes).sort_values(by=['MM:DD:YYYY hh:mm:ss'])
        return singular_df

In [3]:
concat_df = read_csv_files('/Users/kevinmcpherson/github-projects/fed-scripts/data/FED001') # Point to the specific FED folder (i.e., 1, 2, 3 etc)
concat_df.to_csv('/Users/kevinmcpherson/github-projects/fed-scripts/concat_data/FED001_concat.csv') # Saves the file to a csv file of your naming

In [4]:
start_time = input("Enter the date and start time (YYYY-MM-DD hh:mm:ss, e.g., 2023-05-01 10:00:00): ")
start_time = dt.strptime(start_time, '%Y-%m-%d %H:%M:%S')
print(start_time)

2022-06-16 12:00:00


### Make time column datetime object

In [5]:
concat_df['MM:DD:YYYY hh:mm:ss'] = pd.to_datetime(concat_df['MM:DD:YYYY hh:mm:ss'])
concat_df['Time Delta'] = concat_df['MM:DD:YYYY hh:mm:ss'] - start_time
concat_df.head()

,MM:DD:YYYY hh:mm:ss,Library_Version,Session_type,Device_Number,Battery_Voltage,Motor_Turns,FR,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,Time Delta
0,2022-06-16 14:14:23,1.12.0,ClosedEcon,13,4.01,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.22,0 days 02:14:23
1,2022-06-16 14:14:41,1.12.0,ClosedEcon,13,4.01,NaN,1,Left,Left,1,1,0,0,NaN,NaN,0.10,0 days 02:14:41
2,2022-06-16 14:14:49,1.12.0,ClosedEcon,13,4.01,1.0,1,Pellet,Left,1,1,1,1,6.91,NaN,NaN,0 days 02:14:49
3,2022-06-16 15:27:35,1.12.0,ClosedEcon,13,4.00,NaN,1,Left,Left,2,1,1,0,NaN,NaN,0.40,0 days 03:27:35
4,2022-06-16 15:27:36,1.12.0,ClosedEcon,13,4.00,1.0,1,Pellet,Left,2,1,2,1,0.52,4367.0,NaN,0 days 03:27:36


### For now, find out where the file starts and ends

In [6]:

print('The file starts at', concat_df['MM:DD:YYYY hh:mm:ss'].iloc[0])
print('The file ends at', concat_df['MM:DD:YYYY hh:mm:ss'].iloc[-1])

print('The file starts at', concat_df['Time Delta'].iloc[0])
print('The file ends at', concat_df['Time Delta'].iloc[-1])

The file starts at 2022-06-16 14:14:23
The file ends at 2022-07-06 09:49:18
The file starts at 0 days 02:14:23
The file ends at 19 days 21:49:18


In [7]:
def convert_to_hours(delta):
    total_seconds = delta.total_seconds()
    hours = str(int(total_seconds // 3600)).zfill(2)
    minutes= str(int((total_seconds % 3600) // 60)).zfill(2)
    seconds = str(int(total_seconds % 60)).zfill(2)
    return f"{hours}:{minutes}:{seconds}"

def extract_hours(time_delta):
    hours_int = int(time_delta.split(':')[0])
    return hours_int

In [8]:
concat_df['Time Delta Reformatted'] = concat_df['Time Delta'].apply(convert_to_hours)
concat_df['Hours Since Start'] = concat_df['Time Delta Reformatted'].apply(extract_hours)
concat_df.tail() # Save the data as a CSV

,MM:DD:YYYY hh:mm:ss,Library_Version,Session_type,Device_Number,Battery_Voltage,Motor_Turns,FR,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,Time Delta,Time Delta Reformatted,Hours Since Start
1127,2022-07-06 09:49:08,1.12.0,ClosedEcon_PR1,1,3.81,NaN,5,Left,Left,1007,4,117,4,NaN,NaN,0.12,19 days 21:49:08,477:49:08,477
1128,2022-07-06 09:49:10,1.12.0,ClosedEcon_PR1,1,3.80,NaN,5,Left,Left,1008,4,117,4,NaN,NaN,0.07,19 days 21:49:10,477:49:10,477
1129,2022-07-06 09:49:13,1.12.0,ClosedEcon_PR1,1,3.81,NaN,5,Left,Left,1009,4,117,4,NaN,NaN,0.40,19 days 21:49:13,477:49:13,477
1130,2022-07-06 09:49:14,1.12.0,ClosedEcon_PR1,1,3.81,NaN,5,Left,Left,1010,4,117,4,NaN,NaN,0.57,19 days 21:49:14,477:49:14,477
1131,2022-07-06 09:49:18,1.12.0,ClosedEcon_PR1,1,3.81,2.0,5,Pellet,Left,1010,4,118,5,0.48,40.0,NaN,19 days 21:49:18,477:49:18,477


In [9]:
# def create_bins(time_period_var, time_delta_series):
#     
#     hour_end = int(time_delta_series.loc[-1].split(':')[0])
#     for i in range(0, hour_end, time_period_var):


bins = [i for i in range(0,int(concat_df['Time Delta Reformatted'].iloc[-1].split(':')[0])+24,24)]

labels = [f'{i}-{i+24}hrs' for i in range(0,int(concat_df['Time Delta Reformatted'].iloc[-1].split(':')[0]),24)]

concat_df['Bin'] = pd.cut(concat_df['Hours Since Start'], bins, labels=labels, right=True)

In [10]:
concat_df.head()

,MM:DD:YYYY hh:mm:ss,Library_Version,Session_type,Device_Number,Battery_Voltage,Motor_Turns,FR,Event,Active_Poke,Left_Poke_Count,Right_Poke_Count,Pellet_Count,Block_Pellet_Count,Retrieval_Time,InterPelletInterval,Poke_Time,Time Delta,Time Delta Reformatted,Hours Since Start,Bin
0,2022-06-16 14:14:23,1.12.0,ClosedEcon,13,4.01,NaN,1,Right,Left,0,1,0,0,NaN,NaN,0.22,0 days 02:14:23,02:14:23,2,0-24hrs
1,2022-06-16 14:14:41,1.12.0,ClosedEcon,13,4.01,NaN,1,Left,Left,1,1,0,0,NaN,NaN,0.10,0 days 02:14:41,02:14:41,2,0-24hrs
2,2022-06-16 14:14:49,1.12.0,ClosedEcon,13,4.01,1.0,1,Pellet,Left,1,1,1,1,6.91,NaN,NaN,0 days 02:14:49,02:14:49,2,0-24hrs
3,2022-06-16 15:27:35,1.12.0,ClosedEcon,13,4.00,NaN,1,Left,Left,2,1,1,0,NaN,NaN,0.40,0 days 03:27:35,03:27:35,3,0-24hrs
4,2022-06-16 15:27:36,1.12.0,ClosedEcon,13,4.00,1.0,1,Pellet,Left,2,1,2,1,0.52,4367.0,NaN,0 days 03:27:36,03:27:36,3,0-24hrs


### Make bins based on those times

In [11]:
# bins = [10, 11, 12, 13, 14]
# labels = ['hour 1', 'hour 2', 'hour 3', 'hour 4']
# events = ['Left', 'Right', 'Pellet', 'LeftWithPellet', 'RightWithPellet']

### Aggregate based on the results

In [12]:
concat_df.loc[(concat_df['Event'] == 'Right') | (concat_df['Event'] == 'Left') | (concat_df['Event'] == 'Pellet') | (concat_df['Event']=='LeftWithPellet') | (concat_df['Event']=='RightWithPellet'), 'Event Binary'] = 1   

In [13]:
nt_df = pd.DataFrame(concat_df[['Bin', 'Event', 'Event Binary']].groupby(['Bin','Event']).sum())
nt_df.head()

Event Binary
Bin     Event                        
0-24hrs Left                   1686.0
        LeftWithPellet            1.0
        Pellet                  142.0
        Right                    55.0
        RightWithPellet           0.0

In [14]:
tdf = nt_df.groupby(['Bin','Event'])['Event Binary'].first().unstack(fill_value=1).rename_axis(None)
tdf['Pokes Per Pellet'] = tdf['Left']/tdf['Pellet']
tdf['Pellet (kcal)'] = tdf['Pellet']*0.066
tdf['Max FR'] = concat_df.groupby('Bin')['FR'].max()
tdf['Max FR Timestamp'] = concat_df.groupby('Bin')['MM:DD:YYYY hh:mm:ss','FR'].max()['MM:DD:YYYY hh:mm:ss']

tdf

/var/folders/ns/ryjnxrk12jj4nhs8gn4qnfh40000gn/T/ipykernel_72077/3200597951.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  tdf['Max FR Timestamp'] = concat_df.groupby('Bin')['MM:DD:YYYY hh:mm:ss','FR'].max()['MM:DD:YYYY hh:mm:ss']


Event,Left,LeftWithPellet,Pellet,Right,RightWithPellet,Pokes Per Pellet,Pellet (kcal),Max FR,Max FR Timestamp
0-24hrs,1686.0,1.0,142.0,55.0,0.0,11.873239,9.372,46,2022-06-17 12:56:10
24-48hrs,2374.0,0.0,174.0,8.0,0.0,13.643678,11.484,52,2022-06-18 12:41:52
48-72hrs,1214.0,1.0,176.0,14.0,0.0,6.897727,11.616,31,2022-06-19 12:59:35
72-96hrs,1354.0,0.0,153.0,23.0,0.0,8.849673,10.098,39,2022-06-20 12:46:52
96-120hrs,971.0,0.0,154.0,10.0,1.0,6.305195,10.164,30,2022-06-21 12:34:00
120-144hrs,1105.0,1.0,150.0,8.0,0.0,7.366667,9.900,33,2022-06-22 12:49:02
144-168hrs,1077.0,0.0,129.0,7.0,0.0,8.348837,8.514,35,2022-06-23 12:19:41
168-192hrs,691.0,340.0,124.0,10.0,0.0,5.572581,8.184,22,2022-06-24 10:53:45
192-216hrs,44.0,4.0,15.0,5.0,2.0,2.933333,0.990,8,2022-06-25 05:32:05
216-240hrs,1.0,0.0,1.0,1.0,0.0,1.000000,0.066,1,2022-06-25 20:53:50


In [48]:
concat_df.groupby('Bin')['FR','MM:DD:YYYY hh:mm:ss'].agg({'FR': ['max', 'first']})

/var/folders/ns/ryjnxrk12jj4nhs8gn4qnfh40000gn/T/ipykernel_72077/1736964904.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  concat_df.groupby('Bin')['FR','MM:DD:YYYY hh:mm:ss'].agg({'FR': ['max', 'first']})


FR      
           max first
Bin                 
0-24hrs     46     1
24-48hrs    52     1
48-72hrs    31     1
72-96hrs    39     1
96-120hrs   30     1
120-144hrs  33     1
144-168hrs  35     1
168-192hrs  22     1
192-216hrs   8     1
216-240hrs   1     1
240-264hrs   4     1
264-288hrs  13     1
288-312hrs  12     1
312-336hrs   4     1
336-360hrs  27     1
360-384hrs  22     1
384-408hrs  29     1
408-432hrs  23     1
432-456hrs  19     1
456-480hrs  29     1